In [3]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from scipy.interpolate import griddata
import time
from itertools import product, combinations
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from plotting import newfig, savefig
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.gridspec as gridspec
from matplotlib import rc

np.random.seed(1234)


In [4]:
def plot_solution(X_star, u_star, index):
    
    lb = X_star.min(0)
    ub = X_star.max(0)
    nn = 200
    x = np.linspace(lb[0], ub[0], nn)
    y = np.linspace(lb[1], ub[1], nn)
    X, Y = np.meshgrid(x,y)
    
    U_star = griddata(X_star, u_star.flatten(), (X, Y), method='cubic')
    
    plt.figure(index)
    plt.pcolor(X,Y,U_star, cmap = 'jet')
    plt.colorbar()
    
    
def axisEqual3D(ax):
    extents = np.array([getattr(ax, 'get_{}lim'.format(dim))() for dim in 'xyz'])
    sz = extents[:,1] - extents[:,0]
    centers = np.mean(extents, axis=1)
    maxsize = max(abs(sz))
    r = maxsize/4
    for ctr, dim in zip(centers, 'xyz'):
        getattr(ax, 'set_{}lim'.format(dim))(ctr - r, ctr + r)

In [5]:
def load_model(model, path, device):
    model.load_state_dict(torch.load(path, map_location=device))


In [21]:
from train_heat import PhysicsInformedNN, load_data
device = 'cpu'
layers = 8
nIter = 50000  # original niter is 200000
lr = 0.001
batch = 10000
hidden_dim = 512
N_train = 1000 #5000
N_test = 1000
optim_method = "adam"       

collc_xt, collc_z, data_xt, data_z, test_xt, test_z = load_data(device)
data_xt = data_xt[:N_train]
data_z = data_z[:N_train]
test_xt = test_xt[:N_test]
test_z = test_z[:N_test]

pinn = PhysicsInformedNN(data_xt, data_z, layers, device, optim_method, hidden_dim, lr).to(device) 
path = './model/20220501_1330'
load_model(pinn, path, device)


u, f_u = pinn.forward(data_xt)
f_u = f_u.detach().cpu().numpy()
print(f_u.shape)
n_inf = max(np.abs(f_u))
n_2 = 1./np.sqrt(N_train)*np.linalg.norm(f_u, ord=2)

print(n_inf, n_2, n_inf/n_2)


(1000,)
0.07649499 0.040392213617623955 1.8938053879263268


In [17]:
# for heat equation
from train_heat import PhysicsInformedNN, load_data
from torch.autograd import grad
# device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device = 'cpu'
collc_xt, collc_z, data_xt, data_z, test_xt, test_z = load_data(device)

optim_method = "adam"
layers = 8
nIter = 50000  # original niter is 200000
lr = 0.001
batch = 10000

# Load Model
pinn = PhysicsInformedNN(data_xt, data_z, layers, device, optim_method, lr).to(device) 
print(pinn)
path = './model/20220426_1719'
load_model(pinn, path, device)

# Make prediction with trained model
test_xt = test_xt[:2]
u_pred, _ = pinn.forward(test_xt)
u_pred = u_pred.squeeze()
print(u_pred.shape, test_xt.shape)

Du = grad(u_pred, test_xt, grad_outputs=torch.ones_like(torch.Tensor(u_pred)), create_graph=True, retain_graph=True)
u_x = Du[0][:,0]
Du_x = grad(u_x, test_xt, grad_outputs=torch.ones_like(torch.Tensor(u_pred)), create_graph=True, retain_graph=True)
u_xx = Du_x[0][:,0]
Du_xx = grad(u_xx, test_xt, grad_outputs=torch.ones_like(torch.Tensor(u_pred)), create_graph=True, retain_graph=True)
u_xxx = Du_xx[0][:,0]
Du_xxx = grad(u_xxx, test_xt, grad_outputs=torch.ones_like(torch.Tensor(u_pred)), create_graph=True, retain_graph=True)
u_xxxx = Du_xxx[0][:,0]

print(torch.linalg.norm(u_x, 2))
print(torch.linalg.norm(u_xx, 2))
print(torch.linalg.norm(u_xxx, 2))
print(torch.linalg.norm(u_xxxx, 2))
    

PhysicsInformedNN(
  (input_layer): Linear(in_features=2, out_features=20, bias=True)
  (hidden_layers): ModuleList(
    (0): Linear(in_features=20, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=20, bias=True)
    (2): Linear(in_features=20, out_features=20, bias=True)
    (3): Linear(in_features=20, out_features=20, bias=True)
    (4): Linear(in_features=20, out_features=20, bias=True)
    (5): Linear(in_features=20, out_features=20, bias=True)
    (6): Linear(in_features=20, out_features=20, bias=True)
    (7): Linear(in_features=20, out_features=20, bias=True)
  )
  (ouput_layer): Linear(in_features=20, out_features=1, bias=True)
)
torch.Size([2]) torch.Size([2, 2])
tensor(4.2734, grad_fn=<CopyBackwards>)
tensor(11.5721, grad_fn=<CopyBackwards>)
tensor(147.6738, grad_fn=<CopyBackwards>) tensor([121.8490, -83.4289], grad_fn=<SelectBackward0>)
tensor(334.2044, grad_fn=<CopyBackwards>)


/home/quantumiracle/anaconda3/envs/x/lib/python3.7/site-packages/torch/nn/functional.py:1933: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
